In [1]:
from lxml import html
import requests, re, math
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
import random

In [2]:
dictionary =  {'Name': ["Clay Sowell", "Test 2", "Test 3"], 'AB': [57, 3, 4], 'Hits': [23, 1, 2], '2B':[1, 1, 0], '3B':[1, 0, 0], 'HR':[6, 0, 2], 'K':[9, 2, 2], 'Walks':[2, 0, 1], 'HBP':[4, 0, 0]}
df = pd.DataFrame(dictionary)
df.head()

,Name,AB,Hits,2B,3B,HR,K,Walks,HBP
0,Clay Sowell,57,23,1,1,6,9,2,4
1,Test 2,3,1,1,0,0,2,0,0
2,Test 3,4,2,0,0,2,2,1,0


In [3]:
df2 = df
df2['1B'] = 0
df2.head()

,Name,AB,Hits,2B,3B,HR,K,Walks,HBP,1B
0,Clay Sowell,57,23,1,1,6,9,2,4,0
1,Test 2,3,1,1,0,0,2,0,0,0
2,Test 3,4,2,0,0,2,2,1,0,0


In [4]:
curr_thesh = 0
for i, row in df.iterrows():
    curr_thresh = 0
    df2['1B'][i] = df['Hits'][i] - df['2B'][i] - df['3B'][i] - df['HR'][i] + df['Walks'][i] + df['HBP'][i]    
    curr_thresh = ((df['AB'][i]-df['Hits'][i]-df['Walks'][i]-df['HBP'][i])/1.0)/df['AB'][i]
    df2.loc[df2.index[i], 'out_threshold'] = curr_thresh
    curr_thresh += df['1B'][i]/df['AB'][i]
    df2.loc[df2.index[i], '1B_threshold'] = curr_thresh
    curr_thresh += df['2B'][i]/df['AB'][i]
    df2.loc[df2.index[i], '2B_threshold'] = curr_thresh
    curr_thresh += df['3B'][i]/df['AB'][i]
    df2.loc[df2.index[i], '3B_threshold'] = curr_thresh
    curr_thresh += df['HR'][i]/df['AB'][i]
    df2.loc[df2.index[i], 'HR_threshold'] = curr_thresh

df2.head()

<ipython-input-4-84ff83191272>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df2['1B'][i] = df['Hits'][i] - df['2B'][i] - df['3B'][i] - df['HR'][i] + df['Walks'][i] + df['HBP'][i]


,Name,AB,Hits,2B,3B,HR,K,Walks,HBP,1B,out_threshold,1B_threshold,2B_threshold,3B_threshold,HR_threshold
0,Clay Sowell,57,23,1,1,6,9,2,4,21,0.491228,0.859649,0.877193,0.894737,1.0
1,Test 2,3,1,1,0,0,2,0,0,0,0.666667,0.666667,1.000000,1.000000,1.0
2,Test 3,4,2,0,0,2,2,1,0,1,0.250000,0.500000,0.500000,0.500000,1.0


In [12]:
def run_poss():
    game_state = dict({'1B': 0, '2B': 0, '3B':0, 'Outs': 0, 'Runs_Jumbos':0, 'Tufts Batter Next':0, 'Inning':1})
    for i in range(1,10):
        # For each inning, reset the out count and bases
        game_state['Outs'] = 0
        game_state['1B'] = 0
        game_state['2B'] = 0
        game_state['3B'] = 0
        
        while (game_state['Outs'] < 3):
            rand = (random.randint(1,10000))/10000.0
            if (rand < df2['out_threshold'][game_state['Tufts Batter Next']]):
                game_state['Outs'] = game_state['Outs']+1
            elif (rand < f['1B_threshold'][game_state['Tufts Batter Next']]):
                game_state['Runs_Jumbos'] += game_state['3B']
                game_state['3B'] = game_state['2B']
                game_state['2B'] = game_state['1B']
                game_state['1B'] = 1
            elif (rand < df2['2B_threshold'][game_state['Tufts Batter Next']]):
                game_state['Runs_Jumbos'] += game_state['3B']
                game_state['Runs_Jumbos'] += game_state['2B']
                game_state['3B'] = game_state['1B']
                game_state['2B'] = 1
                game_state['1B'] = 0
            elif (rand < df2['3B_threshold'][game_state['Tufts Batter Next']]):
                game_state['Runs_Jumbos'] += game_state['3B']
                game_state['Runs_Jumbos'] += game_state['2B']
                game_state['Runs_Jumbos'] += game_state['1B']
                game_state['3B'] = 1
                game_state['2B'] = 0
                game_state['1B'] = 0
            # Else - home run
            else:
                game_state['Runs_Jumbos'] += game_state['3B']
                game_state['Runs_Jumbos'] += game_state['2B']
                game_state['Runs_Jumbos'] += game_state['1B']
                game_state['1B'] = 0
                game_state['2B'] = 0
                game_state['3B'] = 0
                game_state['Runs_Jumbos'] += 1

            game_state['Tufts Batter Next'] = (game_state['Tufts Batter Next']+1)%(len(df2))

    return game_state['Runs_Jumbos']

In [21]:
# Define Variables
num_iterations = 1000

def run_montecarlo(num_iter):
    runs = []
    for i in range(num_iter):
        runs.append(run_poss())
    
    return ("Result of montecarlo simulations with batting order is an average of " + str(round(sum(runs)/float(len(runs)), 2)) + " runs.")
            
print(run_montecarlo(num_iterations))

Result of montecarlo simulations with batting order is an average of 22.97 runs.


In [ ]:
# TODO - Have one inning, need to run 9 times and aggregate runs scored


        

### Notes:

I assume that Jumbos start each inning -> not hard to change, but while loop continuing game should be altered

Similarly, we should add a way to pick diff pitchers for each time, would have to factor them in somehow to weight performance... or a way for the user to change opposition batting order...
